In [1]:
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)
  

In [2]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        candidate_pos.extend("party")
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        retList = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            retList.append(key)
            if i > number:
                break     
        return retList
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

        

In [3]:
import csv
import re
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

counter = 1
with open('/home/raj/UB_Stuff/CSE_635/Final_Data/India/Distributed_data_from_India.csv','r') as csvinput:
    with open('/home/raj/UB_Stuff/CSE_635/Final_Data/India/Distributed_data_from_India_textRank.csv', 'w') as csvoutput:
        writer = csv.writer(csvoutput, lineterminator='\n')
        reader = csv.reader(csvinput)

        all = []
        row = next(reader)
        row.append('textrank')
        all.append(row)

        for row in reader:
            try:
                if(counter == 361) :
                    break
                searchTerm = row[0]
                print(searchTerm)
                #Get Keywords
                doc = nlp(searchTerm)
                content = doc._.coref_resolved
                tr4w = TextRank4Keyword()
                tr4w.analyze(content, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
                keyWords = tr4w.get_keywords(6)          
                row.append(keyWords)
                all.append(row)
                counter+=1
            except Exception as e:
                print(e)
                pass
        writer.writerows(all)

The Jammu Kashmir police Saturday booked 52 persons in Rajouri district for ignoring administrative orders to deposit their licensed weapons in police stations in run up to the upcoming parliamentary polls. Rajouri, which falls in Jammu parliamentary segment, is going to polls in the first phase on April 11. The district magistrates across the state have directed people to deposit their licensed weapons in local police stations as part of the efforts to ensure free and fair polls. An order in this regard was issued by Rajouri district magistrate on March 1 and all the licensed weapon holders were asked to deposit their weapons in local police stations in view of the forthcoming parliamentary polls. Although majority of licensed weapon holders deposited their weapons in police stations but some people paid no attention to the order even after police served them reminder through notices, a police spokesman said. Taking note of it, he said various cases under relevant sections of law have

police - 3.747719833638583
idols - 3.212828836693609
time - 2.1191586469086467
Dalit - 2.046491147741148
girl - 2.003463919413919
year - 1.9869242063492063
act - 1.6793749306249306
miscreants - 1.5680237108640518
A 35 year old man committed suicide by hanging with the ceiling at his Shyam Nagar EWS under Habibganj police station area on Tuesday. Police said that the deceased Anil Raikwar was found hanging and was rushed to a nearby hospital where he was declared brought dead. In the initial investigation, police found that the deceased committed suicide late in the night and his body was spotted in the early morning hours. During the investigation police have not found any suicide note or any reason behind the suicide. The family members failed to provide any reason behind the suicide. It was found that the deceased used to remain depressed for the past few days but the reason behind the behavior remained unknown. He was posted with the Income Tax department and other details are under

girl - 2.906293392777907
victim - 2.5429210023459654
McDonald - 1.9176513514406213
Law - 1.7531732464395509
security - 1.7086916966696615
guards - 1.6749810292520242
police - 1.6215600189912704
Shubham - 1.5328157753842278
A cashier of a private firm was looted of Rs 9 lakh cash in broad daylight near Chouhan Plaza under Chowni Police station on Tuesday afternoon. The victim identified as Ashish Sharma worked in a company that deals in tobacco. He was on his way to deposit Rs 9 lakh in the Union Bank Supela, when the loot happened, Additional Superintendent of Police ASP City Rohit Kumar Jha said. The incident happened near Chouhan Plaza close to Karbala Dargah around 2 pm, police said. The victim who was on a bike was accosted by three bike borne assailants coming from behind. When the victim stopped the bike, the youths took out a gun and fired shots in the air, police said. The assailants then looted the cash at the gunpoint and fled on the bike. Victim then informed the police, who

JeI - 4.358480935030459
leaders - 2.7406720626451495
Pakistan - 2.7072925801753835
outfit - 2.0419518027827626
Jamaat - 2.0180423545118313
group - 2.017460613136959
e - 1.7905188777462149
ban - 1.7677479821415503
A case has been registered against Bhartiya Janta Party BJP leader Jaya Prada under section 171 G of the Indian Penal Code IPC for making personal statements against Samajwadi Party leader Azam Khan and Bahujan Samaj Party chief Mayawati for violating the Model Code of Conduct MCC . The case has been registered against the BJP leader for allegedly saying, Considering the comments Azam Khan makes against me, Mayawati you must think unki x ray jaisi aankhein aapke upar bhi kaha kaha daal kar dekhengi his X rays like eyes will also stare at you . The case was registered on April 20 based on the comments made by Jaya Prada during an election rally on April 18. Earlier this month, without naming Jaya Prada, Azam Khan while addressing an election rally in Rampur had stated, ...You g

intruders - 4.409157098423854
Army - 3.2517641196804816
Pakistan - 2.8839258058043225
fire - 2.5723704026660985
LoC - 2.5100213517996948
attack - 2.0500568715685676
Indian - 1.9402882003546482
troops - 1.699454314230496
Karachi Unidentified gunmen shot dead at least 14 passengers after forcing them to disembark from a bus on a highway in Pakistan's restive Balochistan province, a media report said Thursday. According to Levies sources, around 15 to 20 unidentified gunmen in camouflage uniform reportedly stopped five or six buses travelling between Karachi and Gwadar. They stopped a bus on the Makran Coastal Highway in Balochistan's Ormara area, checked the identity cards of passengers and offloaded about 16 of them. At least 14 were shot dead, while two passengers managed to escape the massacre and made it to the nearest Levies check post. They were shifted to a hospital for treatment, the Dawn newspaper reported. Levies and other law enforcement personnel reached the spot and launched

...

Chouhan - 2.193371735389124
T - 2.1793998368007985
Chief - 2.1495488620573973
Minister - 2.086027395076379
MP - 2.070208015183289
officials - 1.9243893306687436
Department - 1.778428099575208
BJP - 1.7621147151091034
Srinagar, March 27 Militants on Wednesday shot and killed a civilian in Jammu and Kashmir's Shopian district, police said. Related Three criminals killed in UP Tanveer Ahmed Dar, 24, who belonged to Bemnipora village, succumbed to critical injuries instantly, a police officer said.
police - 1.270920634920635
Bemnipora - 1.2050287698412698
Dar - 1.2017906746031746
Wednesday - 1.1926160714285712
village - 1.171787698412698
civilian - 1.1656488095238093
Jammu - 1.1614494047619046
Militants - 1.0358551587301585
The motorcycle allegedly used as getaway vehicle by the killers of former BJP MLA Jayanti Bhanushali was found abandoned by a head constable in Kutch, but he did not put it on police records and handed it over to the accused, not knowing it would be used in the crime , 

Kakkar - 3.9671436565353386
Nath - 2.907869843876459
Kamal - 2.5455599564246145
officials - 2.3979228517863236
T - 2.3051975937507616
OSD - 2.172140494036165
Tax - 1.9163370147752947
Income - 1.9001019665893537
The attack is seen as the Maoists' way of issuing a warning. New Delhi A devastating Maoist attack just two days before the national elections killed the BJP lawmaker of Chhattisgarh's Dantewada on Tuesday. His driver and three security personnel also died in the attack. Bhima Mandavi, 40, was out for campaigning. The police said he had gone to the area against their advice. Prime Minister Narendra Modi has condemned the attack and said it was deeply anguishing . The Election Commission has sought a report, but said polling in the state will be held as per schedule. Dantewada votes on April 11 in the first phase. Elections will also be held in the state in two other phases, on April 18 and 23.
attack - 2.0112520833333334
Dantewada - 1.8566412273242627
Chhattisgarh - 1.5930636479

traders - 3.042045969090984
Surat - 2.366848809947768
March - 2.1125582169566544
Zoo - 2.075841448449038
Bear - 1.9466052692016977
saris - 1.741585032438901
Army - 1.5304914173789173
Soldiers - 1.49702741529304
Students protesting outside the senate meeting venue Download The Times of India News App for Latest City SURAT A group of students held protest outside the Veer Narmad South Gujarat University VSNGU senate hall on Wednesday and shouted slogans against vice chancellor Shivendra Gupta The VNSGU senate meeting was held at the administrative building in which two third members remained absent. Majority of the senators remained absent to register protest against the alleged dictatorial administration by the VC and his aides, sources at the university informed.The group of students started protest soon after the senate meeting began at 12 noon and continued till the senate meeting ended at 3.30 pm. The students initially shouted slogans on stairs of the administrative building and de

Dinesh - 5.531979456473793
Preeti - 2.670695727552915
Kumar - 2.4653956288981993
Delhi - 2.2099075536302197
temple - 2.149499934765937
Annu - 1.9536504713767884
Pintoo - 1.8378491985234637
relationship - 1.8177031713198075
Pictures provided by Income Tax Sources of cash recovered during raid at residential premises of Prateek Joshi in Bhopal, Madhya Pradesh. I T searches are underway at 50 locations including Indore, Bhopal, Goa and Delhi. pic.twitter.com TAMe4J1Nii ANI ANI April 7, 2019 Indore Visuals from official premises of Praveen Kakkar, OSD to Madhya Pradesh CM, where income tax officials are conducting a raid. pic.twitter.com fWoOS4qT4o ANI ANI April 7, 2019 Delhi Income Tax department official searches car of RK Miglani, in Green Park. pic.twitter.com 2uJFkUHaK1 ANI ANI April 7, 2019 In a major crackdown by the Income Tax department, the residence and official premises of Madhya Pradesh Chief Minister Kamal Nath s close aide Praveen Kakkar were raided by the officials of the d

Delhi - 3.071594897151959
president - 2.366214984253774
University - 2.001562574408608
Tyagi - 1.5847638318636705
ABVP - 1.551404356509272
hostels - 1.482847222222222
K - 1.4249410370834141
DUSU - 1.3777242691583207
Chennai, April 1 IANS A large amount of cash, neatly packed in cartons and gunny bags, was found by Income Tax IT Department officials in a cement warehouse, owned by a DMK leader, in Tamil Nadu s Vellore district, an official said on Monday. According to a senior IT department official, the cash was shifted from Kingston College belonging to Duraimurugan Educational Trust, Vellore to the cement warehouse of the DMK leader, considered close to party treasurer Duraimurugan, late on March 29. The official said the cash was shifted due to the botched up operation by the state police and revenue authorities and was found on Monday morning in the cement warehouse by the IT investigation wing. All the cash was stuffed in cartons and gunny bags neatly packed for distribution to di

In [9]:
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

text = r'The ruling Trinamool Congress and the opposition BJP, who are ready to battle it out in the Lok Sabha polls in West Bengal, are preferring turncoats over their tried and tested leaders. There is discontent over candidate selection within both the parties dominating the poll scenario in the state. Senior TMC leader and its South Dinajpur district chief Biplab Mitra said there is anger among grass roots level workers of the party over giving tickets to newcomers and ignoring the old timers. The TMC is going solo and has declared names of candidates for all 42 Lok Sabha seats. The BJP declared its first list of 28 contestants in the state on Thursday. The two parties defend their decision, saying winnability was of prime consideration for them. However, the TMC feels it is the best way to quell the infighting in the party, whereas for the BJP, fielding turncoats had become a compulsion as it did not have enough candidates to assure its victory. We have party leaders and workers who are working very hard. We had given them tickets in panchayat and assembly polls. But in case of fighting Lok Sabha polls, we don t have enough candidates who can put up a fight and win, state BJP chief Dilip Ghosh said last Friday. Out of 18 new faces in TMC s list, seven are those who have either switched over from the Congress or the Left parties in the past few years. Two of the TMC s sitting MPs in north and south Bengal Dasarth Tirkey and Sunil Mondal who have been renominated this time, used to be MLAs of Left Front constituents RSP and Forward Bloc, respectively. BJP s list has six candidates who were earlier either with the TMC or the CPI M . It also includes the name of an ex IPS officer, who was once considered close to TMC chief Mamata Banerjee. A TMC leader told PTI on condition of anonymity that infighting is one of the biggest problems being faced by the party this time. There are so many factions in every district that it was next to impossible to bring all the disgruntled leaders on the same page...That is why in at least seven seats which are considered tough, we have pitted those who have come from other parties so that opposing factions will at least unite under their umbrellas to help us win, he said. Out of the seven candidates who have either switched over from the Congress or the Left parties in the past few years, three have been nominated from Bolpur, Medinipur and Coochbehar which the TMC had won last time. In Malda North, Raiganj, Jangipur and Murshidabad seats, which were won by the Congress and the CPI M last time, the TMC has nominated MLAs and MPs who had switched over from the Congress in the last two years. In Cooch Behar, former Left Front minister Paresh Chandra Adhikary was nominated as only a strong candidate can ensure in retaining the seat in the face of factionalism and resurgent BJP there, said a senior TMC MLA from north Bengal. The reason is the same behind nominating former state Congress president Manas Bhunia from Mednipur and Asit Mal from Bolpur. Former Congress MP Mausam Benazir Noor, who crossed over to the TMC, and has been nominated from Malda North seat told PTI, I had joined TMC as I feel only it can fight against communal BJP in the state. On the other hand, sitting TMC MP from Bolpur, Anupam Hazra has switched over to the BJP and has been fielded from Jadavpur seat in the city. In Murshidabad, TMC youth leader and key organisational man Shamik Hossain was overlooked in favour of former Congress MLA Abu Taher, who had switched over only last year. The situation is more difficult in the BJP, which had won two seats in 2014 and now targeting to win 23 seats in the state. The BJP has emerged as the main opposition party after the 2018 panchayat polls. The party in its first list has five candidates who had recently defected to it from the TMC and one from the CPI M . Former IPS officer Bharati Ghosh who recently joined BJP will be contesting Ghatal Lok Sabha seat on the party s ticket. During her stint as the Jhargram SP, she was considered to be the defacto TMC district president. Former TMC MLA Arjun Singh, who recently switched over to the BJP, has been pitted against TMC s Dinesh Trivedi in Barrackpore.Singh had quit the TMC recently after the party denied him a ticket for the Barrackpore seat. Former TMC MP Soumitra Khan has been given BJP ticket from his Bishnupur seat, while former CPI M MLA Khagen Murmu will contest on a BJP ticket from Malda North. Former TMC leader Sreerupa Mitra Chowdhury, who had contested polls on that party s ticket from New Delhi, is the BJP candidate from Malda South seat this time, while TMC youth leader Nishith Pramanik, who was expelled last year, has been fielded from Cooch Behar. The BJP has fielded another TMC turncoat Bolpur MP Anupam Hazra from Jadavpur. BJP leader Mukul Roy, once TMC s number two, defended the saffron party s decision on fielding turncoats. Anybody who wants to fight against TMC misrule in Bengal is welcome, Roy said. BJP workers had staged protest in Cooch Behar and Malda North against the party s decision to nominate TMC and CPI M leaders instead of local party leaders. Several BJP leaders, including its vice president Raj Kamal Pathak, have submitted their resignation and protests have been organised in various parts of the state over candidate selection. The seven phase polling in the state will begin on April 11.'
doc = nlp(text)
content = doc._.coref_resolved


tr4w = TextRank4Keyword()
tr4w.analyze(content, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
keyWords = tr4w.get_keywords(4)


TMC - 6.255037380297151
BJP - 6.032471931766575
Congress - 3.0193872097785546
Left - 1.840893907568426
Bolpur - 1.8341633621370117
Malda - 1.7586502069116925
